# This file is used to train the weights that is used in our CNN
We kept it as a ipynb file, as it was trained in google colab.

We start by connecting the notebook to Google Drive

In [11]:
from google.colab import drive
drive.mount('/content/drive/')
import numpy as np

from unet import *
from augmentation import *
from training_helpers import test_weights

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


### Once we have imported all necessary files we start by making a model and print a summary of how it is constructed

In [4]:
model = get_unet()
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 256, 256, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 256, 256, 64) 1792        input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 256, 256, 64) 36928       conv2d_1[0][0]                   
__________________________________________________________________________________________________
max_pooling2d_1 (MaxP

/content/unet.py:52: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("co...)`
  model = Model(input = inputs, output = conv10)


### We now declare variables to tell the CNN where it can retrieve both original and ground truth images, as well as where to store the weights file

In [0]:
#using google drive to fetch images
drive_dir = "drive/My Drive/"
#variables used to find folders of images
train_dir = drive_dir+"train/"
weights_dir = train_dir+"weights/"
image_folder = 'originals'
gt_folder = 'whites'
#this is the name the weights file will get once it is trained
weights_filename ='weights.hdf5'

### We declare variables that is used to create a larger image dataset through augmentation. Note that as the image dataset is less suited for augmentation we do not generate as large variety of images as we might have with another dataset.

In [0]:
#augmentation variables
generator_variables = dict(rotation_range=0.2,
                           width_shift_range=0.1,
                           height_shift_range=0.1,
                           zoom_range=0.2,
                           horizontal_flip=True,
                           vertical_flip = False,
                           fill_mode='reflect')
#variables used to decide how long the model is trained
batch_size = 2
steps_per_epoch = 300
epochs=10

### We then train the model. The images are retrieved from the drive. With 300 steps per epoch, batch size of 2 and 10 epochs it takes approximately 30 minutes to train the model in Colab using GPU accelerator. We can also train the model over again several times.

In [15]:

train_generator = trainGenerator(batch_size,train_dir,image_folder,gt_folder,generator_variables)
#model.load_weights(weights_dir + weights_filename)
checkpoint = ModelCheckpoint(weights_dir + weights_filename, monitor='loss', verbose=1, save_best_only=True)
model.fit_generator(train_generator,steps_per_epoch=steps_per_epoch,epochs=epochs, callbacks=[checkpoint])

Epoch 1/10
Found 50 images belonging to 1 classes.
Found 50 images belonging to 1 classes.
300/300 [==============================] - 195s 650ms/step - loss: 0.0897 - acc: 0.9667

Epoch 00001: loss improved from inf to 0.08966, saving model to drive/My Drive/train/weights/weights.hdf5
Epoch 2/10
300/300 [==============================] - 192s 640ms/step - loss: 0.0950 - acc: 0.9653

Epoch 00002: loss did not improve from 0.08966
Epoch 3/10
300/300 [==============================] - 191s 638ms/step - loss: 0.0879 - acc: 0.9679

Epoch 00003: loss improved from 0.08966 to 0.08785, saving model to drive/My Drive/train/weights/weights.hdf5
Epoch 4/10
300/300 [==============================] - 194s 646ms/step - loss: 0.0872 - acc: 0.9682

Epoch 00004: loss improved from 0.08785 to 0.08716, saving model to drive/My Drive/train/weights/weights.hdf5
Epoch 5/10
300/300 [==============================] - 193s 642ms/step - loss: 0.0840 - acc: 0.9692

Epoch 00005: loss improved from 0.08716 to 0.08

### Once we have  a trained model, we use methods in training_helpers.py to save predictions on a few images, as well as calculate metrics such as accuracy, and f1 to give a sense of how well the model performs.

In [17]:
#variables to tell where test images are located and where prediction images should be stored
test_dir = train_dir+"testing/"
test_image_dir = test_dir+"test_images/"
test_gt_dir = test_dir+"test_gt/"
prediction_dir = test_dir+"predictions/"
#we then declare variables for the standardized names of the images in the test folders as well as how many of them we want
test_image_name = "test%d.png"
test_gt_name = "test_gt%d.png"
first_image = 1
last_image = 6

#We then run a method that stores predictions and returns different metrics
accuracy, recall, precision, f1 = test_weights(weights_dir+weights_filename,prediction_dir, test_image_dir, 
                                               test_image_name,test_gt_dir, test_gt_name, first_image,last_image,
                                               treshold=0.2)

print("Accuracy: "+str(accuracy)+"\n"+
     "Recall: "+str(recall)+"\n"+
     "Precision: "+str(precision)+"\n"+
     "f1: "+str(f1)+"\n")

/content/unet.py:52: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("co...)`
  model = Model(input = inputs, output = conv10)


KeyboardInterrupt: ignored